# 准备

## 基础设置

In [ ]:
GSE_Train = "GSE63990" # 用于训练的数据集
Skip_Paint = False # 是否绘制不必要的绘图


## 库 导入

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import warnings
import GEOparse
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report, accuracy_score, roc_auc_score
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LinearRegression, RidgeClassifier, Ridge, Lasso, LassoCV, LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC

## 数据 导入

In [ ]:
GPL571 = GEOparse.get_GEO(geo="GPL571", destdir="./datasets", silent=True)

In [ ]:
gse = GEOparse.get_GEO(geo=GSE_Train, destdir="./datasets", silent=True)
gpls = gse.metadata['platform_id']
gpl = GEOparse.get_GEO(geo=gpls[0], destdir="./datasets", silent=True)
gse_csv = pd.read_csv('./datasets/' + GSE_Train + '.csv')
gse_csv.head() # 预览数据

In [ ]:
gse_csv.isnull().sum() 

In [ ]:
gse_csv['infection_status'].value_counts() # 看一下感染这个列的分布

## 函数 准备

In [ ]:
def is_prime_factors_only(n, allowed_factors):
    for factor in allowed_factors:
        while n % factor == 0:
            n //= factor
    return n == 1

def get_allowed_numbers(max_n=25,allowed_factors=[2,3,5]):
    result = []
    for i in range(1, max_n + 1):
        if is_prime_factors_only(i, allowed_factors):
            result.append(i)
    return result

def is_all_in_list(list,n=25,allowed_factors=[2,3,5]):
    for i in list:
        if i not in get_allowed_numbers(n,allowed_factors):
            return False
    return list

In [ ]:
def plot_score_distribution(ax, y_score, final_target):                                 # 这个函数负责处理分数分布图这一个小图
    ax.scatter(range(len(y_score)), y_score, c=final_target, cmap='bwr', alpha=0.5)     # 画散点图，x轴是样本序号，y轴是分数，颜色是感染情况，颜色映射是蓝白红，透明度是0.5
                                                                                        # x是样本序号：分开画，免得都黏在一起了，透明度的设置是一样的原因。
    ax.set_title('分数分布图')
    ax.set_xlabel('样本序号')
    ax.set_ylabel('分数')
    plt.rcParams['font.sans-serif']=['SimHei']                                          # 中文乱码
    plt.rcParams['axes.unicode_minus']=False                                            # 负号乱码
    labels = ['细菌感染', '病毒感染']                                                   # 图例标签，这里是感染情况，以后如果换了的话得改。 #todo 将这个改为传入参数
    ax.legend(labels, loc='upper right')                                                # 图例
    ax.plot([0, len(y_score)], [0.5, 0.5], color='black', lw=1, linestyle='--')         # 以0.5为界，画一条虚线，表示分数大于0.5的是细菌感染，小于0.5的是病毒感染，这个以后也得改
                                                                                        # 因为不同的模型（如linear regression 和 ridge regression）的判断标准不一样，所以这个界限也不一样。
                                                                                        # 现在的做法是直接在y_score 上做手脚，还是……蛮不优雅的，但是先这样吧。
                                                                                        # todo 将这个改为传入参数

def plot_roc_curve(ax, y_score, final_target):
    fpr, tpr, thresholds = roc_curve(final_target, y_score)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax.set_xlabel('False Positive Rate 假阳性率')
    ax.set_ylabel('True Positive Rate 真阳性率')
    ax.set_title('ROC 曲线（受试者工作特征曲线）')
    ax.legend(loc="lower right")

def plot_pr_curve(ax, y_score, final_target):
    precision, recall, thresholds = precision_recall_curve(final_target, y_score)
    ax.plot(recall, precision, color='darkorange', lw=2, label='PR curve')
    ax.set_xlabel('Recall 召回率')
    ax.set_ylabel('Precision 准确率')
    ax.set_title('PR 曲线（准确率-召回率曲线）')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.legend(loc="lower left")

def plot_confusion_matrix(ax, y_score, final_target):
    y_pred = np.where(y_score > 0.5, 1, 0)
    cnf_matrix = confusion_matrix(final_target, y_pred)
    ax.imshow(cnf_matrix, interpolation='nearest',cmap=plt.cm.Blues)
    ax.set_title('Confusion Matrix 混淆矩阵')
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['病毒感染', '细菌感染'], rotation=45)
    plt.yticks(tick_marks, ['病毒感染', '细菌感染'])
    thresh = cnf_matrix.max() / 2.
    for i in range(cnf_matrix.shape[0]):
        for j in range(cnf_matrix.shape[1]):
            plt.text(j, i, cnf_matrix[i, j],horizontalalignment="center",color="white" if cnf_matrix[i, j] > thresh else "black",fontsize=20)
    plt.tight_layout()
    plt.ylabel('True label 感染情况')
    plt.xlabel('Predicted label 预测情况')

def plot_all(score, target, title="分数分布图、ROC曲线、PR曲线和混淆矩阵", feature=None, weight=None):
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))                                                     # 规定画布大小为1200*1200px，分成2*2的4个小图
    fig.suptitle(title, fontsize=16)                                                                    # 设置标题
    plot_score_distribution(axs[0,0], score, target)                                                    # 第一个小图（左上角）是分数分布图 这里的axs[0,0]是指第一行第一列的小图，注意计数是从0开始的。
    plot_roc_curve(axs[0,1], score, target)                                                             # 第二个小图（右上角）是ROC曲线
    plot_pr_curve(axs[1,0], score, target)                                                              # 第三个小图（左下角）是PR曲线
    plot_confusion_matrix(axs[1,1], score, target)                                                      # 第四个小图（右下角）是混淆矩阵
    if feature or weight:                                                                               # 如果有feature或者weight的话，就在图的下方加上文字
        try:
            # add margin at bottom
            fig.subplots_adjust(bottom=0.12)                                                            # 调整图的下边距，这样文字就不会挡住图了
            # add feature and weight at the bottom
            fig.text(0.5, 0.04, '使用feature:'+str(feature), ha='center', va='bottom', fontsize=12)     # 在图的下方加上文字，这里的0.5,0.04是指文字的位置，ha='center'是指文字居中，va='bottom'是指文字在底部，fontsize=12是指文字大小
            fig.text(0.5, 0.02, '对应权重:'+str(weight), ha='center', va='bottom', fontsize=8)          # 同上
        except:
            pass                                                                                        # 如果出错了就算了
    return fig, axs

def testfeature(feature,weight,data,gpl=gpl,title="分数分布图、ROC曲线、PR曲线和混淆矩阵"):
    ID = []
    for i in range(len(feature)):
        ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0])
    ID = np.array(ID).reshape(-1)
    testdata = data[ID]
    n = len(testdata)
    y_score = np.zeros(n)
    y_pred = np.zeros(n)
    for i in range(len(testdata)):
        y_score [i] = np.dot(testdata.iloc[i],weight)
        y_pred [i] = np.where(y_score[i] > 0.5, 1, 0)
    try:
        target = data['infection_status'].values
    except:
        target = gse_csv['infection_status']
    acc = accuracy_score(target, y_pred)
    print("Accuracy: ", acc)
    fig, axs = plot_all(y_score, target, title, feature=feature, weight=weight)

def testfeaturewithtimes(feature,weight,data,times=1,gpl=gpl,title="分数分布图、ROC曲线、PR曲线和混淆矩阵"):
    ID = []
    for i in range(len(feature)):
        ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0])
    ID = np.array(ID).reshape(-1)
    testdata = data[ID]
    n = len(testdata)
    y_score = np.zeros(n)
    y_pred = np.zeros(n)
    for i in range(len(testdata)):
        y_score [i] = np.dot(testdata.iloc[i],weight)
        # 结果除以times
        y_score [i] = y_score [i] / times
        y_pred [i] = np.where(y_score[i] > 0.5, 1, 0)
    try:
        target = data['infection_status'].values
    except:
        target = gse_csv['infection_status']
    acc = accuracy_score(target, y_pred)
    print("Accuracy: ", acc)
    fig, axs = plot_all(y_score, target, title, feature=feature, weight=weight)

def testfeaturewithtimesandtarget(feature,weight,data,target,times=1,gpl=gpl,title="分数分布图、ROC曲线、PR曲线和混淆矩阵"):
    ID = []
    for i in range(len(feature)):
        ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0])
    ID = np.array(ID).reshape(-1)
    testdata = data[ID]
    n = len(testdata)
    y_score = np.zeros(n)
    y_pred = np.zeros(n)
    for i in range(len(testdata)):
        y_score [i] = np.dot(testdata.iloc[i],weight)
        # 结果除以times
        y_score [i] = y_score [i] / times
        y_pred [i] = np.where(y_score[i] > 0.5, 1, 0)
    acc = accuracy_score(target, y_pred)
    print("Accuracy: ", acc)
    fig, axs = plot_all(y_score, target, title, feature=feature, weight=weight)


In [ ]:
def find_best_n(ax,X_train, y_train, X_test, y_test, method, model, max_features=20):
    train_acc,test_acc,train_roc,test_roc = [],[],[],[]
    for i in range(1, max_features):
        # reselect the features
        reselect = SelectKBest(score_func=method, k=i)
        reselect.fit(X_train, y_train)
        model.fit(X_train.iloc[:, reselect.get_support(indices=True)], y_train) # use the selected features to train the data
        y_pred = model.predict(X_test.iloc[:, reselect.get_support(indices=True)])
        train_acc.append(model.score(X_train.iloc[:, reselect.get_support(indices=True)], y_train))
        test_acc.append(model.score(X_test.iloc[:, reselect.get_support(indices=True)], y_test))
        train_roc.append(roc_auc_score(y_train, model.predict(X_train.iloc[:, reselect.get_support(indices=True)])))
        test_roc.append(roc_auc_score(y_test, model.predict(X_test.iloc[:, reselect.get_support(indices=True)])))
    # ax.plot(range(1, max_features), train_acc, label='train_acc')
    ax.plot(range(1, max_features), test_acc, label='test_acc')
    # ax.plot(range(1, max_features), train_roc, label='train_roc')
    ax.plot(range(1, max_features), test_roc, label='test_roc')
    ax.set_title('使用'+str(model) + '和' + method.__name__ + '选择特征')
    plt.rcParams['font.sans-serif'] = ['SimHei']
    ax.legend()
    # grid
    ax.grid()
    # mark the maximum value
    ax.scatter(test_acc.index(max(test_acc)) + 1, max(test_acc), marker='o', color='b')
    ax.text(test_acc.index(max(test_acc)) + 1, max(test_acc), str(test_acc.index(max(test_acc)) + 1) + ', ' + str('%.2f%%' % (max(test_acc) * 100)))
    ax.scatter(test_roc.index(max(test_roc)) + 1, max(test_roc), marker='o', color='r')
    ax.text(test_roc.index(max(test_roc)) + 1, max(test_roc), str(test_roc.index(max(test_roc)) + 1) + ', ' + str('%.2f%%' % (max(test_roc) * 100)))
    # y is 0.5 to 1
    ax.set_ylim(0.5, 1)

    select = SelectKBest(score_func=method, k=test_acc.index(max(test_acc)) + 1).fit(X_train, y_train)
    X_train_new = select.transform(X_train)
    X_test_new = select.transform(X_test)
    model.fit(X_train_new, y_train)
    y_pred = model.predict(X_test_new)
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的准确率为：', model.score(X_test_new, y_test))
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的AUC为：', roc_auc_score(y_test, y_pred))
    # # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的权重为：', model.coef_)
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的截距为：', model.intercept_)
    # get column names
    gene_id = X_train.columns[select.get_support(indices=True)]
    gene_symbol = [gpl.table[gpl.table['ID'] == i]['Gene Symbol'].values[0] for i in gene_id]
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的特征为：', gene_symbol)
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的特征权重为：', model.coef_)
    
    n = test_acc.index(max(test_acc)) + 1
    best_acc = max(test_acc)

    return best_acc, n, gene_symbol, model.coef_


In [ ]:
def find_best_n(ax,X_train, y_train, X_test, y_test, method, model, max_features=20):
    train_acc,test_acc,train_roc,test_roc = [],[],[],[]
    for i in range(1, max_features):
        if method == 'mannual':
            reselect = X_train.iloc[:, :i]
            model.fit(reselect, y_train) # use the selected features to train the data
            y_pred = model.predict(X_test.iloc[:, :i])
            train_acc.append(model.score(reselect, y_train))
            test_acc.append(model.score(X_test.iloc[:, :i], y_test))
            train_roc.append(roc_auc_score(y_train, model.predict(reselect)))
            test_roc.append(roc_auc_score(y_test, model.predict(X_test.iloc[:, :i])))
        else:
            reselect = SelectKBest(score_func=method, k=i)
            reselect.fit(X_train, y_train)
            model.fit(X_train.iloc[:, reselect.get_support(indices=True)], y_train) # use the selected features to train the data
            y_pred = model.predict(X_test.iloc[:, reselect.get_support(indices=True)])
            train_acc.append(model.score(X_train.iloc[:, reselect.get_support(indices=True)], y_train))
            test_acc.append(model.score(X_test.iloc[:, reselect.get_support(indices=True)], y_test))
            train_roc.append(roc_auc_score(y_train, model.predict(X_train.iloc[:, reselect.get_support(indices=True)])))
            test_roc.append(roc_auc_score(y_test, model.predict(X_test.iloc[:, reselect.get_support(indices=True)])))
    # ax.plot(range(1, max_features), train_acc, label='train_acc')
    ax.plot(range(1, max_features), test_acc, label='test_acc')
    # ax.plot(range(1, max_features), train_roc, label='train_roc')
    ax.plot(range(1, max_features), test_roc, label='test_roc')
    if method == 'mannual':
        ax.set_title('使用'+str(model) + '按照表达比选择特征')
    else:
        ax.set_title('使用'+str(model) + '和' + method.__name__ + '选择特征')
    plt.rcParams['font.sans-serif'] = ['SimHei']
    ax.legend()
    # grid
    ax.grid()
    # mark the maximum value
    ax.scatter(test_acc.index(max(test_acc)) + 1, max(test_acc), marker='o', color='b')
    ax.text(test_acc.index(max(test_acc)) + 1, max(test_acc), str(test_acc.index(max(test_acc)) + 1) + ', ' + str('%.2f%%' % (max(test_acc) * 100)))
    ax.scatter(test_roc.index(max(test_roc)) + 1, max(test_roc), marker='o', color='r')
    ax.text(test_roc.index(max(test_roc)) + 1, max(test_roc), str(test_roc.index(max(test_roc)) + 1) + ', ' + str('%.2f%%' % (max(test_roc) * 100)))
    # y is 0.5 to 1
    ax.set_ylim(0.5, 1)

    if method != 'mannual':
        select = SelectKBest(score_func=method, k=test_acc.index(max(test_acc)) + 1).fit(X_train, y_train)
        X_train_new = select.transform(X_train)
        X_test_new = select.transform(X_test)
        model.fit(X_train_new, y_train)
        y_pred = model.predict(X_test_new)
        gene_id = X_train.columns[select.get_support(indices=True)]
    else:
        X_train_new = X_train.iloc[:, :test_acc.index(max(test_acc)) + 1]
        X_test_new = X_test.iloc[:, :test_acc.index(max(test_acc)) + 1]
        model.fit(X_train_new, y_train)
        y_pred = model.predict(X_test_new)
        gene_id = X_train_new.columns
    gene_symbol = [gpl.table[gpl.table['ID'] == i]['Gene Symbol'].values[0] for i in gene_id]
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的特征为：', gene_symbol)
    # print('使用' + str(model) + '和' + method.__name__ + '选择特征后的特征权重为：', model.coef_)
    
    n = test_acc.index(max(test_acc)) + 1
    best_acc = max(test_acc)

    return best_acc, n, gene_symbol, model.coef_


In [ ]:
def predict(model, X, time=None):
    # if not coef: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
    if time is not None:
        coef = model.coef_.ravel() * time
        coef = np.round(coef)
        time = float(time)
    else:
        time = 1
        time = float(time)
        coef = model.coef_.ravel()
    if model.__class__.__name__ == 'LinearRegression':
        y_pred = float(np.dot(X, coef))/time + model.intercept_ 
    elif model.__class__.__name__ == 'RidgeClassifier':
        y_pred = np.dot(X, coef)/time + model.intercept_ + 0.5# 以0为分界（？
    elif model.__class__.__name__ == 'LogisticRegression':
        z = np.dot(X, coef)/time + model.intercept_
        y_pred = 1 / (1 + np.exp(-z))
        # y_pred = np.dot(X, model.coef_.ravel()) + model.intercept_
    elif model.__class__.__name__ == 'LinearSVC':
        y_pred = np.dot(X, coef)/time + model.intercept_
    elif model.__class__.__name__ == 'NuSVC':
        y_pred = np.dot(X, coef) / time + model.intercept_ + 0.5 # 以0为分界（？
    else:
        y_pred = model.predict(X)
    return y_pred

In [ ]:
def cal_score(feature, weight, times, data, model, intercept):
    score = []
    indexs = []
    for i in range(len(feature)):
        id = gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0]
        index = data.columns.get_loc(id)
        indexs.append(index)
    for i in range(len(data)):
        # if model is in LinearRegression, LinearSVC
        if model == model.__class__.__name__ == 'LinearRegression' or model.__class__.__name__ == 'LinearSVC':  
            score.append(
                np.dot(
                    data.iloc[i, indexs].values,weight
                    )/times
                )
        elif model == model.__class__.__name__ == 'RidgeClassifier' or model.__class__.__name__ == 'NuSVC':  
            score.append(
                np.dot(
                    data.iloc[i, indexs].values,weight
                    )/times + 0.5
                )
        elif model == model.__class__.__name__ == 'LogisticRegression':
            z = np.dot(
                data.iloc[i, indexs].values,weight
            )/times  + intercept
            
            score.append(1/(1+np.exp(-z)))
        else:
            z = np.dot(
                data.iloc[i, indexs].values,weight
            )/times  + intercept
            score.append(1 / (1 + np.exp(-z)))

    return score

In [ ]:
def test_method(select_method, model, n_feature, X_train, X_test, y_train, y_test):
    success = False
    reselect = SelectKBest(score_func=select_method, k=n_feature)
    reselect.fit(X_train, y_train)
    X_train_new = reselect.transform(X_train)
    X_test_new = reselect.transform(X_test)
    model.fit(X_train_new, y_train)
    y_score= predict(model, X_test_new)
    y_pred = model.predict(X_test_new)
    model_acc = np.mean(np.equal(y_pred, y_test))
    print('使用 ' + select_method.__name__ + ' 选择特征和 ' + str(model) + ' 训练模型，准确率为', '%.2f%%' % (model_acc * 100))
    title = '使用' + select_method.__class__.__name__ + '选择' + str(n_feature) + '个特征的' + model.__class__.__name__ + '的直接运算的预测结果'
    gene_id = X_train.columns[reselect.get_support(indices=True)]
    feature = [gpl.table[gpl.table['ID'] == i]['Gene Symbol'].values[0] for i in gene_id]
    coef = model.coef_

    print('截距：', model.intercept_ , ' 选出基因： ', feature)

    times = np.logspace(1, 8, num=500, base=10)
    accs = []
    good_coef = []
    for time in times:
        y_score = predict(model, X_test_new, time)
        y_pred = np.array([1 if i > 0.5 else 0 for i in y_score], dtype=int)
        accs.append(np.mean(y_pred == y_test))
        if np.mean(y_pred == y_test) == model_acc:
            coef_rounded = np.round(coef.ravel() * time).tolist()
            coef_rounded = [int(i) for i in coef_rounded]
            if is_all_in_list(coef_rounded) and coef_rounded not in good_coef:
                print('coef =', coef_rounded,'time =', time)
                good_coef.append(coef_rounded)
                success = True
        else:
            pass
    plot_all(
        score = y_score,
        target = y_test,
        title = title,
        feature = feature,
        weight = model.coef_.ravel())
    plt.show()

    plt.plot(times, accs)
    plt.xlabel('times')
    plt.ylabel('accuracy')
    plt.title('Relationship between times and accuracy')
    plt.xscale('log')
    plt.show()

    coef = model.coef_

    return coef, reselect

In [ ]:
def plot_boxplot(data_higher, gpl, gse_csv, data_higher_times):
    fig, ax = plt.subplots(1,50, figsize=(50,15))
    for i in range(50):
        sns.boxplot(
            x='infection_status', 
            y=data_higher.iloc[:,i],
            data=gse_csv,
            ax=ax[i],
            )
        ax[i].set_title(gpl.table[gpl.table['ID'] == data_higher.columns[i]]['Gene Symbol'].values[0])

        # 最上面加上对应的倍数
        ax[i].text(0.1, 0.98,
                   str('%.2f' % data_higher_times[i]),
                   transform=ax[i].transAxes,
                   fontsize=16,
                   verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        # 显示优化
        ax[i].set_yscale('log') #对数坐标
        ax[i].set_ylim(10, 40000) # y轴的范围
        if i != 0: # 不显示x，y轴的标签， 不然太挤了没意义
            ax[i].set_xlabel('')
            ax[i].set_yticklabels([])
            ax[i].set_ylabel('')
            ax[i].set_yticks([])
        else:
            ax[i].set_xlabel('')
            ax[i].set_ylabel('表达量')
        ax[i].tick_params(axis='x', rotation=90) # 不然画不下
    plt.show()

In [ ]:
def get_higher_columns(data):
    data_mean = data.groupby('target').mean()
    higher_columns = []
    higher_times = []
    for i in range(data_mean.shape[1]):
        if data_mean.iloc[1,i] > 2*data_mean.iloc[0,i]:
            higher_columns.append(data_mean.columns[i])
            higher_times.append(data_mean.iloc[1,i]/data_mean.iloc[0,i])
    return higher_columns, higher_times

In [ ]:
def sort_higher_columns(data, higher_columns, higher_times):
    # sort the columns and times together
    data_higher_times, data_higher_columns = zip(*sorted(zip(higher_times, higher_columns), reverse=True))
    # convert data_higher_columns to list
    data_higher_columns = list(data_higher_columns)
    # add target column to the top of data_higher_columns
    data_higher_columns.append('target')
    data_higher_columns.append('infection_status')
    # get the new data
    data_higher = data[data_higher_columns]
    return data_higher, data_higher_times

In [ ]:
def print_best_acc(test_acc, n, gene_symbol, coef):
    best_acc = max(test_acc)
    best_i = 0
    i = 0
    print('最好的准确率为：', best_acc)
    for acc in test_acc:
        if acc == best_acc:
            print('='*60)
            print('对应的特征数为：', n[i],'方法为：', methods[i // len(models)].__name__,'特征为：', gene_symbol[i])
            print('模型为：', models[i % len(models)],'特征权重为：', coef[i])
            weights_avg = np.mean(coef[i])
            new_coefs = []
            for j in range(len(coef[i])):
                # new_coefs.append(10*(coef[i][j]/weights_avg)) 保留2位小数
                # new_coefs.append(round(10*(coef[i][j]/weights_avg), 2)) type numpy.ndarray doesn't define __round__ method
                new_coefs.append(np.round(10*(coef[i][j]/weights_avg), 2))
            print('比例大致为：', new_coefs)
        i += 1

# 训练

## 病毒

### 分开病毒与未感染

In [ ]:
# 病毒与未感染的鉴别
viral_data = gse_csv[gse_csv['infection_status'].isin(['viral', 'non-infectious illness', 'bacterial'])]
viral_data['target'] = viral_data['infection_status'].map({'viral': 1, 'non-infectious illness': 0, 'bacterial': 0})
X_train, X_test, y_train, y_test = train_test_split(viral_data.drop(['infection_status'], axis=1), viral_data['infection_status'], test_size=0.2, random_state=0)

### 筛选出 病毒感染中表达量大于等于2倍的非病毒感染表达量的基因

In [ ]:
higher_columns, higher_times = get_higher_columns(viral_data)

根据倍率不同进行排序

In [ ]:
data_higher, data_higher_times = sort_higher_columns(viral_data, higher_columns, higher_times)
X_train, X_test, y_train, y_test = train_test_split(data_higher.drop(['infection_status','target'], axis=1), data_higher['target'], test_size=0.2, random_state=0)

使用 boxplot 对排序后的基因画图，选了前50个

In [ ]:
if Skip_Paint:
    plot_boxplot(data_higher, gpl, gse_csv, data_higher_times)

### 训练不同的模型，使用不同的特征选择方法，选择出特征后，使用不同的方法来训练模型

In [ ]:
methods = [chi2, f_classif, mutual_info_classif,'mannual']
models = [
    # https://scikit-learn.org/stable/modules/linear_model.html
    LinearRegression(
        positive=True
    ),                              # 普通 的 非负
    RidgeClassifier(
    ),                              # 是最小二乘法的正则化版本，明显快于例如 LogisticRegression，该分类器有时称为具有线性内核的最小二乘支持向量机
    LogisticRegression(
    ),                              # 逻辑回归
    Lasso(
        positive=True
    ),                              # 稀疏的 非负
    # SVC(
    #     kernel='linear',
    #     class_weight='balanced'
    # ),                            # 超级超级超级慢 一次5~6分钟
    LinearSVC(
        
    ),
    NuSVC(
        kernel='linear'
    )
]
# methods = [chi2]
# models = [LinearRegression()]
fig, axs = plt.subplots(len(methods), len(models), figsize=(6*len(models), 4*len(methods)))
test_accs = []
best_n = []
rocs = []
gene_symbols = []
coefs = []
for method in methods:
    for model in models:
        # find_best_n(axs[methods.index(method), models.index(model)], X_train, y_train, X_test, y_test, method, model, max_features=6)
        test_acc, n, gene_symbol, coef = find_best_n(
            axs[methods.index(method), models.index(model)], 
            X_train, y_train, X_test, y_test, 
            method, 
            model, 
            max_features=9)
        test_accs.append(test_acc)
        best_n.append(n)
        gene_symbols.append(gene_symbol)
        coefs.append(coef)
plt.show()

获取上面训练的结果 —— 筛选出上面的表中表现的较好的方法

In [ ]:
print_best_acc(test_accs, best_n, gene_symbols, coefs)

In [ ]:
coef, reselect = test_method(
    chi2, 
    LogisticRegression(),
    3, 
    X_train, X_test, y_train, y_test)

## 细菌

In [ ]:
# 病毒与未感染的鉴别
bacterial_data = gse_csv[gse_csv['infection_status'].isin(['viral', 'non-infectious illness', 'bacterial'])]
bacterial_data['target'] = bacterial_data['infection_status'].map({'bacterial': 1, 'non-infectious illness': 0, 'viral': 0})
X_train, X_test, y_train, y_test = train_test_split(bacterial_data.drop(['infection_status'], axis=1), bacterial_data['infection_status'], test_size=0.2, random_state=0)
bacterial_data['infection_status'].value_counts()

In [ ]:
higher_columns, higher_times = get_higher_columns(bacterial_data)

In [ ]:
data_higher, data_higher_times = sort_higher_columns(bacterial_data, higher_columns, higher_times)
X_train, X_test, y_train, y_test = train_test_split(data_higher.drop(['infection_status','target'], axis=1), data_higher['target'], test_size=0.2, random_state=0)

In [ ]:
if Skip_Paint:
    plot_boxplot(data_higher, gpl, gse_csv, data_higher_times)

选

In [ ]:
methods = [chi2, f_classif, mutual_info_classif,'mannual']
models = [
    # https://scikit-learn.org/stable/modules/linear_model.html
    LinearRegression(
        positive=True
    ),                              # 普通最小二乘法 的 非负
    Lasso(
        positive=True
    ),                              # 稀疏的 非负
    RidgeClassifier(
        # alpha=1
    ),                              # 是最小二乘法的正则化版本，明显快于例如 LogisticRegression，该分类器有时称为具有线性内核的最小二乘支持向量机
    LogisticRegression(
        # penalty='l2',
        # C=1
    ),                              # 逻辑回归
    # SVC(
    #     kernel='linear',
    #     class_weight='balanced'
    # ),                            # 超级超级超级慢
    LinearSVC(
        
    ),
    NuSVC(
        kernel='linear'
    )
]
# methods = [chi2]
# models = [LinearRegression()]
fig, axs = plt.subplots(len(methods), len(models), figsize=(6*len(models), 4*len(methods)))
test_accs = []
best_n = []
rocs = []
gene_symbols = []
coefs = []
for method in methods:
    for model in models:
        # find_best_n(axs[methods.index(method), models.index(model)], X_train, y_train, X_test, y_test, method, model, max_features=6)
        test_acc, n, gene_symbol, coef = find_best_n(
            axs[methods.index(method), models.index(model)], 
            X_train, y_train, X_test, y_test, 
            method, 
            model, 
            max_features=10)
        test_accs.append(test_acc)
        best_n.append(n)
        gene_symbols.append(gene_symbol)
        coefs.append(coef)
plt.show()

In [ ]:
print_best_acc(test_accs, best_n, gene_symbols, coefs)

In [ ]:
coef, reselect = test_method(
    chi2, 
    LogisticRegression(),
    4, 
    X_train, X_test, y_train, y_test)

# 测试

In [ ]:
viral_feature =  ['ISG15', 'MX1', 'CD74']
viral_weight =  [11,  8, 24]
viral_times = 84694.59808314576
viral_intercept = -2.8716859
viral_model = LogisticRegression()
bacterial_feature =  ['CD177', 'IL1R2', 'S100A12', 'MMP9']
bacterial_weight = [14, 11, 24,  1]
bacterial_times = 151481.89222583472
bacterial_model = LogisticRegression()
bacterial_intercept = -2.84741179

In [ ]:
from matplotlib import patches


viral_score = cal_score(viral_feature, viral_weight,viral_times, gse_csv, viral_model, viral_intercept)
bacterial_score = cal_score(bacterial_feature, bacterial_weight,bacterial_times, gse_csv, bacterial_model, bacterial_intercept)
y = gse_csv['infection_status'].map({'viral': 1, 'non-infectious illness': 0, 'bacterial': 2})

fig, ax = plt.subplots( 1,1, figsize=(10,10))
ax.scatter(
    viral_score,
    bacterial_score,
    c=y,
    alpha=0.6
)
ax.set_xlabel('viral_score')
ax.set_ylabel('bacterial_score')

# 比例尺保持一致
ax.set_aspect('equal')

# 表明不同颜色意义
ax.legend(
    handles=[
        patches.Patch(color='green', label='病毒'),
        patches.Patch(color='purple', label='非感染'),
        patches.Patch(color='yellow', label='细菌')
    ]
)

# 画y=x
ax.plot(
    [0, 1], [0, 1],
    transform=ax.transAxes,
    ls="--",
    c=".3"
)

In [ ]:
test_data = gse_csv[gse_csv['infection_status'].isin(['viral', 'bacterial'])]
test_data['target'] = test_data['infection_status'].map({'viral': 1, 'bacterial': 0})

viral_score = cal_score(viral_feature, viral_weight,viral_times, test_data, viral_model, viral_intercept)
bacterial_score = cal_score(bacterial_feature, bacterial_weight,bacterial_times, test_data, bacterial_model, bacterial_intercept)

# score is  viral_score - bacterial_score + 0.5
# viral_score,  bacterial_score and score are list.
score = np.array(viral_score) - np.array(bacterial_score) + 0.5

feature = viral_feature + bacterial_feature
weight = viral_weight + (np.array(bacterial_weight) * -1).tolist()

plot_all(
    score, 
    test_data['target'],
    title="分数分布图、ROC曲线、PR曲线和混淆矩阵",
    feature=feature,
    weight=weight
)